Answer 3.

a) They don't output a probability. They only output a classification based on a threshold which gets met based on learned weights that can't be interpreted, so the perceptron can only give us yes or no, and they don't tell us anything intuitive about the system under inspection.

b) You can set the activation function to a logistic function. 

Answer 4.

Logistic activations were attractive because they are continuous and differentiable over the entire real line. They 

Answer 5.

There's the ReLU activation function, the step (heaviside function), the logistic function, and the tanh function. 

Yes. I can draw them.

Answer 6.


1.) With n instances, the shape of the input matrix is n $\times$ 10.    
2.) W is $10 \times 50$ in shape. b is $1 \times 50$    
3.) W is $50 \times 3$ b is $1 \times 3$    
4.) The output matrix has shape $n \times 3$    
5.) $Y = ReLU(ReLU(XW_0+ B_0)W_h+b_h)$


Answer 7.

It takes one neuron in the ouptut layer to classify email as ham or spam. You need 10 neurons to classify the mnist data

To predict housing prices, there are two ways to do this. One is to "categorize" shousing prices into increasingly expensive categories, and this will take $n$ neurons for $n$ levels of expense. For a regression prediction, you only need one neuron.

Answer 8.

Backpropagation is a training algorithm. It efficiently computes gradients w.r.t. all weights automatically. Batches of inputs are sent through the Neural network in multiple passes called epochs. The inputs are passed through all the layers of the network, and individual outputs are saved. Output error is measured, and each output connection's contribution to the error is computed using the chain rule. The error contriburtions from the connections in the layer below are then calculated using chain rule, and a gradient descent step is then used to tweak the weights in the network.  

Answer 9. 

Hyperparameters in a basic MLP:

1.) number of input neurons: can be changed by adding features or by compressing the data    
2.) number of hidden layers    
3.) neurons per (in each) hidden layer    
4.) Hidden layer activation function    
5.) Output actiation    
6.) Loss function    

6.) loss functions include sparse_categorical_crossenropy, mean_squared_error    
    __[Keras loss function documentation](https://keras.io/api/losses/)__

### Decision to Scale

The data was unscaled in a preliminary attempt. I'm leaving the choice here because a <tt> uint8 </tt> takes up $\frac{1}{4}$ the space of a <tt> float</tt>, so the spatial savings might be worth a try without scaling the data.    

Since scaling can lead to improvements, and preliminary models only marginally missed the mark, we're going to retry the same tests without scaling. You can retry the tests without scaling by setting to_scale_data to <tt>False</tt>

In [9]:
to_scale_data = False

In [2]:
from tensorflow import keras

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
if to_scale_data:
    X_train = scaler.fit_transform(X_train, y_train)
    X_test = scaler.fit_transform(X_test, y_test)

In [11]:
def basic_mlp_mnist(n_input_neurons=784, n_hidden_layers=2, l_neurons=[30,30], h_activ="relu", opt="adam", met="accuracy"):
    in_l = keras.layers.Flatten(input_shape=[28,28])
    hidden_ls = [ keras.layers.Dense(l_neurons[i], activation=h_activ) for i in range(n_hidden_layers) ]
    out_l = keras.layers.Dense(10, activation="softmax")
    model = keras.models.Sequential([in_l] + hidden_ls + [out_l] )
    model.compile(loss = "sparse_categorical_crossentropy", optimizer=opt, metrics=[met])
    return model

In [12]:
# function that trains @param mod with a tuple data=(X_train, y_train)
# val: validation set
def train_mlp(data, mod, eps=25, val = None, cbs=None):
    if val:
        if cbs:
            cb_in = cbs if isinstance(cbs, list) else [cbs]
            hist = mod.fit(data[0], data[1], epochs=eps, validation_data=val, callbacks=cb_in)
        else:
            hist = mod.fit(data[0], data[1], epochs=eps, validation_data=val)        
    else:
        hist = mod.fit(data[0], data[1], epochs=eps)
    return hist

In [13]:
model0 = basic_mlp_mnist()
history0 = train_mlp((X_train, y_train), model0)

Epoch 1/25
1875/1875 [==============================] - 2s 1ms/step - loss: 1.5993 - accuracy: 0.6819
Epoch 2/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5338 - accuracy: 0.8677
Epoch 3/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4512 - accuracy: 0.8850
Epoch 4/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4032 - accuracy: 0.8954
Epoch 5/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3798 - accuracy: 0.9014
Epoch 6/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3532 - accuracy: 0.9074
Epoch 7/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3364 - accuracy: 0.9122
Epoch 8/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3061 - accuracy: 0.9191
Epoch 9/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2925 - accuracy: 0.9215
Epoch 10/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.274

In [14]:
model0.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.2272 - accuracy: 0.9448


[0.2272450476884842, 0.9448000192642212]

We can see that on our first attempt, with 2 layers, we got 94.5% accuracy. oof. Let's try adding more layers

In [15]:
model1 = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[30,30,30,30,30])
history1 = train_mlp((X_train, y_train), model1)
model1.evaluate(X_test, y_test)

Epoch 1/25
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7640 - accuracy: 0.8040
Epoch 2/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2960 - accuracy: 0.9170
Epoch 3/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2381 - accuracy: 0.9330
Epoch 4/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2026 - accuracy: 0.9437
Epoch 5/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1795 - accuracy: 0.9484
Epoch 6/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1620 - accuracy: 0.9542
Epoch 7/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1508 - accuracy: 0.9570
Epoch 8/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1382 - accuracy: 0.9605
Epoch 9/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1284 - accuracy: 0.9635
Epoch 10/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.120

[0.15822839736938477, 0.9646999835968018]

In [16]:
model1.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1582 - accuracy: 0.9647


[0.15822839736938477, 0.9646999835968018]

Tantalizingly close. We'll try 10 more epochs

In [17]:
history_1_1 = train_mlp((X_train, y_train), model1, eps=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0660 - accuracy: 0.9805
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0628 - accuracy: 0.9820
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0595 - accuracy: 0.9824
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0646 - accuracy: 0.9811
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0580 - accuracy: 0.9833
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0597 - accuracy: 0.9833
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0560 - accuracy: 0.9837
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0589 - accuracy: 0.9833
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0556 - accuracy: 0.9840
Epoch 10/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.054

In [18]:
model1.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1496 - accuracy: 0.9675


[0.1495937556028366, 0.9674999713897705]

We got some imporvement by simply including more layers. We'll try some best practices here and split our training set into training and validation. We'll use the <tt> train_test_split </tt> and then pass it to our training function.

In [19]:
from sklearn.model_selection import train_test_split
#bp for best practices
#We'll be using default parameters here. We can 
X_train_bp, X_valid, y_train_bp, y_valid = train_test_split(X_train, y_train)

In [20]:
model2 = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[30,30,30,30,30])
history2 = train_mlp((X_train_bp, y_train_bp), model2, val=(X_valid, y_valid))
model2.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.8952 - accuracy: 0.7714 - val_loss: 0.3809 - val_accuracy: 0.8939
Epoch 2/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.3221 - accuracy: 0.9088 - val_loss: 0.3074 - val_accuracy: 0.9129
Epoch 3/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.2559 - accuracy: 0.9290 - val_loss: 0.2500 - val_accuracy: 0.9329
Epoch 4/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.2179 - accuracy: 0.9390 - val_loss: 0.2425 - val_accuracy: 0.9333
Epoch 5/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.1930 - accuracy: 0.9456 - val_loss: 0.2296 - val_accuracy: 0.9377
Epoch 6/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.1780 - accuracy: 0.9501 - val_loss: 0.2162 - val_accuracy: 0.9427
Epoch 7/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.1610 - accuracy: 0.9538 - val_loss: 0.2034 - val_accuracy:

[0.1895788460969925, 0.9549999833106995]

There seems to be more information in the tests than we've gotten so far. We'll try using a deeper neural network

In [21]:
model3 = basic_mlp_mnist( n_hidden_layers=9, l_neurons=[30,30,30,30,30,30,30,30,30])
history3 = train_mlp((X_train_bp, y_train_bp), model3, val=(X_valid, y_valid))
model3.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.5640 - accuracy: 0.8289 - val_loss: 0.3622 - val_accuracy: 0.8963
Epoch 2/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.2635 - accuracy: 0.9268 - val_loss: 0.2870 - val_accuracy: 0.9219
Epoch 3/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.2178 - accuracy: 0.9389 - val_loss: 0.2287 - val_accuracy: 0.9381
Epoch 4/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1897 - accuracy: 0.9468 - val_loss: 0.2171 - val_accuracy: 0.9390
Epoch 5/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1711 - accuracy: 0.9522 - val_loss: 0.2331 - val_accuracy: 0.9416
Epoch 6/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1581 - accuracy: 0.9564 - val_loss: 0.1836 - val_accuracy: 0.9493
Epoch 7/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1446 - accuracy: 0.9590 - val_loss: 0.1972 - val_accuracy:

[0.1729763001203537, 0.9642999768257141]

Both the training and validation errors seem to be decreasing as the number of epochs passed increases. We'll try to improve this model by training the network for more epochs. We'll make sure to use some kind of early stopping to make sure we don't go too far.

In [22]:
model3.save("m3_mnist.h5")

In [23]:
model_3_cb = keras.callbacks.ModelCheckpoint("m3_mnist.h5", save_best_only=True)
m3_hist = train_mlp((X_train_bp, y_train_bp), model3, val=(X_valid, y_valid), cbs = [model_3_cb] )

Epoch 1/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0681 - accuracy: 0.9812 - val_loss: 0.2146 - val_accuracy: 0.9565
Epoch 2/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0637 - accuracy: 0.9814 - val_loss: 0.1831 - val_accuracy: 0.9603
Epoch 3/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0651 - accuracy: 0.9817 - val_loss: 0.1890 - val_accuracy: 0.9612
Epoch 4/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0619 - accuracy: 0.9825 - val_loss: 0.1776 - val_accuracy: 0.9589
Epoch 5/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.0637 - accuracy: 0.9827 - val_loss: 0.1790 - val_accuracy: 0.9593
Epoch 6/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0603 - accuracy: 0.9830 - val_loss: 0.1720 - val_accuracy: 0.9627
Epoch 7/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0551 - accuracy: 0.9841 - val_loss: 0.2141 - val_accuracy:

We might not see any improvement just from the initial epochs or from the inclusion of the validation set, so we'll try to use wider layers. This increases the danger of overfitting, but it's worth a shot anyways.

In [24]:
model4 = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[100,100,100,100,100])
history1 = train_mlp((X_train, y_train), model4)
model4.evaluate(X_test, y_test)

Epoch 1/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4935 - accuracy: 0.8782
Epoch 2/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1940 - accuracy: 0.9435
Epoch 3/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1503 - accuracy: 0.9573
Epoch 4/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1301 - accuracy: 0.9634
Epoch 5/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1120 - accuracy: 0.9679
Epoch 6/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0935 - accuracy: 0.9740
Epoch 7/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0925 - accuracy: 0.9747
Epoch 8/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0828 - accuracy: 0.9776
Epoch 9/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0733 - accuracy: 0.9797
Epoch 10/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.067

[0.20166289806365967, 0.972000002861023]

In [25]:
model4.evaluate(X_test, y_test)

313/313 [==============================] - 0s 2ms/step - loss: 0.2017 - accuracy: 0.9720


[0.20166289806365967, 0.972000002861023]

This model looks promising. We can use a validation set to try to better train a model with this architecture.

In [32]:
model4_bp = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[300,100,100,100,100])
history1 = train_mlp((X_train_bp, y_train_bp), model4_bp,  val=(X_valid, y_valid))
model4_bp.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.5681 - accuracy: 0.8773 - val_loss: 0.2146 - val_accuracy: 0.9371
Epoch 2/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.2004 - accuracy: 0.9421 - val_loss: 0.1971 - val_accuracy: 0.9463
Epoch 3/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.1557 - accuracy: 0.9555 - val_loss: 0.1569 - val_accuracy: 0.9573
Epoch 4/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.1364 - accuracy: 0.9610 - val_loss: 0.1482 - val_accuracy: 0.9602
Epoch 5/25
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1091 - accuracy: 0.9696 - val_loss: 0.1442 - val_accuracy: 0.9613
Epoch 6/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.1004 - accuracy: 0.9723 - val_loss: 0.1418 - val_accuracy: 0.9636
Epoch 7/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.0868 - accuracy: 0.9761 - val_loss: 0.1721 - val_accuracy:

[0.17501336336135864, 0.9764000177383423]

In [31]:
model4_bp.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.2198 - accuracy: 0.9708


[0.21979230642318726, 0.97079998254776]

We seem close. We're going to try to use a smaller learning rate to finish up the neural net.

In [36]:
q_opt = keras.optimizers.Adam(learning_rate = 0.0005)
model4_bp = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[300,300,100,100,100], opt=q_opt)
history1 = train_mlp((X_train_bp, y_train_bp), model4_bp,  val=(X_valid, y_valid))
model4_bp.evaluate(X_test, y_test)


Epoch 1/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.6850 - accuracy: 0.8756 - val_loss: 0.2670 - val_accuracy: 0.9265
Epoch 2/25
1407/1407 [==============================] - 6s 5ms/step - loss: 0.1897 - accuracy: 0.9447 - val_loss: 0.1647 - val_accuracy: 0.9521
Epoch 3/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1442 - accuracy: 0.9586 - val_loss: 0.1851 - val_accuracy: 0.9504
Epoch 4/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1269 - accuracy: 0.9631 - val_loss: 0.1540 - val_accuracy: 0.9584
Epoch 5/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0994 - accuracy: 0.9708 - val_loss: 0.1405 - val_accuracy: 0.9618
Epoch 6/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0900 - accuracy: 0.9739 - val_loss: 0.1409 - val_accuracy: 0.9646
Epoch 7/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0780 - accuracy: 0.9775 - val_loss: 0.1397 - val_accuracy:

[0.19260898232460022, 0.9757999777793884]

In [37]:
model4_bp_cb = keras.callbacks.ModelCheckpoint("model4_bp.h5", save_best_only=True)
history4_mp = train_mlp((X_train_bp, y_train_bp), model4_bp,  val=(X_valid, y_valid), cbs=model4_bp_cb )
model4_bp = keras.models.load_model("model4_bp.h5")
model4_bp.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0143 - accuracy: 0.9958 - val_loss: 0.1656 - val_accuracy: 0.9787
Epoch 2/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0214 - accuracy: 0.9950 - val_loss: 0.1364 - val_accuracy: 0.9789
Epoch 3/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0259 - accuracy: 0.9941 - val_loss: 0.1620 - val_accuracy: 0.9795
Epoch 4/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0139 - accuracy: 0.9969 - val_loss: 0.1769 - val_accuracy: 0.9753
Epoch 5/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0188 - accuracy: 0.9958 - val_loss: 0.1776 - val_accuracy: 0.9761
Epoch 6/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0175 - accuracy: 0.9959 - val_loss: 0.1782 - val_accuracy: 0.9765
Epoch 7/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.0194 - accuracy: 0.9954 - val_loss: 0.1701 - val_accuracy:

[0.13464485108852386, 0.978600025177002]

In [38]:
model4_bp_cb = keras.callbacks.ModelCheckpoint("model4_bp.h5", save_best_only=True)
history4_mp = train_mlp((X_train_bp, y_train_bp), model4_bp, eps=50, val=(X_valid, y_valid), cbs=model4_bp_cb )
model4_bp = keras.models.load_model("model4_bp.h5")
model4_bp.evaluate(X_test, y_test)

Epoch 1/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0185 - accuracy: 0.9956 - val_loss: 0.1523 - val_accuracy: 0.9779
Epoch 2/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0142 - accuracy: 0.9962 - val_loss: 0.1674 - val_accuracy: 0.9768
Epoch 3/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0156 - accuracy: 0.9962 - val_loss: 0.1748 - val_accuracy: 0.9759
Epoch 4/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0175 - accuracy: 0.9959 - val_loss: 0.1591 - val_accuracy: 0.9762
Epoch 5/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0163 - accuracy: 0.9963 - val_loss: 0.1836 - val_accuracy: 0.9749
Epoch 6/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0204 - accuracy: 0.9958 - val_loss: 0.1940 - val_accuracy: 0.9770
Epoch 7/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0164 - accuracy: 0.9961 - val_loss: 0.1695 - val_accuracy:

[0.16765132546424866, 0.9761999845504761]

In [39]:
model4_bp.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1677 - accuracy: 0.9762


[0.16765132546424866, 0.9761999845504761]

In [58]:

q_opt = keras.optimizers.Adam(learning_rate = 0.00005)


model4_bp = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[300,300,100,100,100], opt=q_opt, h_activ="tanh")
history1 = train_mlp((X_train_bp, y_train_bp), model4_bp, val=(X_valid, y_valid), eps=50)
model4_bp.evaluate(X_test, y_test)


Epoch 1/50
1407/1407 [==============================] - 4s 3ms/step - loss: 0.7347 - accuracy: 0.7972 - val_loss: 0.3987 - val_accuracy: 0.8860
Epoch 2/50
1407/1407 [==============================] - 5s 3ms/step - loss: 0.3229 - accuracy: 0.9062 - val_loss: 0.2993 - val_accuracy: 0.9093
Epoch 3/50
1407/1407 [==============================] - 6s 5ms/step - loss: 0.2475 - accuracy: 0.9262 - val_loss: 0.2559 - val_accuracy: 0.9240
Epoch 4/50
1407/1407 [==============================] - 6s 4ms/step - loss: 0.2045 - accuracy: 0.9384 - val_loss: 0.2395 - val_accuracy: 0.9277
Epoch 5/50
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1785 - accuracy: 0.9459 - val_loss: 0.2182 - val_accuracy: 0.9335
Epoch 6/50
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1565 - accuracy: 0.9533 - val_loss: 0.2042 - val_accuracy: 0.9388
Epoch 7/50
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1397 - accuracy: 0.9570 - val_loss: 0.1941 - val_accuracy:

[0.13775146007537842, 0.9666000008583069]

In [26]:
model5 = basic_mlp_mnist( n_hidden_layers=3, l_neurons=[100,100,100])
history5 = train_mlp((X_train, y_train), model5)
model5.evaluate(X_test, y_test)

Epoch 1/25
1875/1875 [==============================] - 4s 2ms/step - loss: 1.0485 - accuracy: 0.8665
Epoch 2/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2389 - accuracy: 0.9352
Epoch 3/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1735 - accuracy: 0.9499
Epoch 4/25
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1456 - accuracy: 0.9585
Epoch 5/25
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1303 - accuracy: 0.9628
Epoch 6/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1135 - accuracy: 0.9677
Epoch 7/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1004 - accuracy: 0.9711
Epoch 8/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0916 - accuracy: 0.9739
Epoch 9/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0869 - accuracy: 0.9747
Epoch 10/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.079

[0.17026759684085846, 0.9733999967575073]

In [27]:
model5.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1703 - accuracy: 0.9734


[0.17026759684085846, 0.9733999967575073]

In [28]:
model6 = basic_mlp_mnist( n_hidden_layers=2, l_neurons=[300,300])
history6 = train_mlp((X_train, y_train), model6)
model6.evaluate(X_test, y_test)

Epoch 1/25
1875/1875 [==============================] - 6s 3ms/step - loss: 1.6232 - accuracy: 0.8958
Epoch 2/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2161 - accuracy: 0.9451
Epoch 3/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1636 - accuracy: 0.9559
Epoch 4/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1515 - accuracy: 0.9588
Epoch 5/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1408 - accuracy: 0.9622
Epoch 6/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1219 - accuracy: 0.9683
Epoch 7/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1089 - accuracy: 0.9715
Epoch 8/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1011 - accuracy: 0.9740
Epoch 9/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0925 - accuracy: 0.9768
Epoch 10/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.085

[0.32435357570648193, 0.9731000065803528]

In [29]:
model6 = basic_mlp_mnist( n_hidden_layers=2, l_neurons=[300,50])
history6 = train_mlp((X_train, y_train), model6)
model6.evaluate(X_test, y_test)

Epoch 1/25
1875/1875 [==============================] - 5s 3ms/step - loss: 1.1915 - accuracy: 0.7500
Epoch 2/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3701 - accuracy: 0.9054
Epoch 3/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2324 - accuracy: 0.9407
Epoch 4/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1783 - accuracy: 0.9519
Epoch 5/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1475 - accuracy: 0.9607
Epoch 6/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1244 - accuracy: 0.9667
Epoch 7/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1072 - accuracy: 0.9708
Epoch 8/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0921 - accuracy: 0.9756
Epoch 9/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0906 - accuracy: 0.9758
Epoch 10/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.081

[0.2814026474952698, 0.9675999879837036]